# 01 - Policy Experimentation with OPA

Welcome to interactive policy experimentation with ACGS-2! This notebook guides you through:

1. **Connecting to OPA** - Setting up communication with the policy engine
2. **Basic Policy Queries** - Evaluating your first policies
3. **Role-Based Access Control** - Understanding RBAC patterns
4. **Debugging Policies** - Using denial reasons for troubleshooting
5. **Batch Evaluation** - Testing multiple scenarios efficiently
6. **Visualization** - Graphing policy decisions

## Prerequisites

Before running this notebook, ensure OPA is running:

```bash
# From project root
docker compose up opa -d

# Verify OPA is healthy
curl http://localhost:8181/health
```

---

## 1. Setup and Imports

First, let's import the required libraries and configure the environment for Docker compatibility.

In [ ]:
# Set headless backend BEFORE importing matplotlib (required for Docker)
import os  # noqa: I001

os.environ["MPLBACKEND"] = "Agg"

# Standard library
from typing import Any

# Data manipulation and visualization
import matplotlib.pyplot as plt
import pandas as pd
import requests
import seaborn as sns
from requests.exceptions import RequestException

# Enable inline plots
%matplotlib inline

# Set visualization style
sns.set_theme(style="whitegrid")

## 2. Connect to OPA

The Open Policy Agent (OPA) exposes a REST API on port 8181. We'll create helper functions to interact with it.

In [ ]:
# OPA URL configuration
# - Docker: uses internal network name 'opa'
# - Local: uses localhost
OPA_URL = os.getenv("OPA_URL", "http://localhost:8181")

In [ ]:
def check_opa_health() -> bool:
    """Check if OPA is running and healthy."""
    try:
        response = requests.get(f"{OPA_URL}/health", timeout=5)
        if response.status_code == 200:
            return True
        else:
            return False
    except RequestException:
        return False


# Verify OPA connection
opa_healthy = check_opa_health()

In [ ]:
def evaluate_policy(policy_path: str, input_data: dict[str, Any]) -> dict[str, Any]:
    """Evaluate an OPA policy with the given input data.

    Args:
        policy_path: Path to the policy rule (e.g., 'hello/allow')
        input_data: Dictionary of input values for the policy

    Returns:
        The policy evaluation result

    Raises:
        RequestException: If OPA is unreachable or returns an error
    """
    response = requests.post(
        f"{OPA_URL}/v1/data/{policy_path}",
        json={"input": input_data},
        timeout=10,
    )
    response.raise_for_status()
    return response.json()


def safe_evaluate(policy_path: str, input_data: dict[str, Any]) -> dict[str, Any] | None:
    """Evaluate a policy with error handling.

    Returns None if evaluation fails (instead of raising an exception).
    """
    try:
        return evaluate_policy(policy_path, input_data)
    except RequestException:
        return None

## 3. Basic Policy Queries

Let's start with the "hello world" policy from Example 01. This policy implements simple role-based access control:

- **Admins**: Can perform any action
- **Developers**: Can only read resources
- **Others**: Denied by default

In [ ]:
# Query 1: Admin user trying to delete a resource
admin_delete = {
    "user": {"role": "admin"},
    "action": "delete",
    "resource": "policy",
}

result = safe_evaluate("hello/allow", admin_delete)
if result:
    allowed = result.get("result", False)

In [ ]:
# Query 2: Developer user trying to read a resource
developer_read = {
    "user": {"role": "developer"},
    "action": "read",
    "resource": "policy",
}

result = safe_evaluate("hello/allow", developer_read)
if result:
    allowed = result.get("result", False)

In [ ]:
# Query 3: Developer user trying to delete a resource (should be denied!)
developer_delete = {
    "user": {"role": "developer"},
    "action": "delete",
    "resource": "policy",
}

result = safe_evaluate("hello/allow", developer_delete)
if result:
    allowed = result.get("result", False)

## 4. Debugging with Denial Reasons

When a request is denied, it's helpful to know **why**. The hello policy includes `denial_reasons` that explain the cause of denial.

In [ ]:
def explain_decision(input_data: dict[str, Any]) -> None:
    """Evaluate a policy and explain the decision."""
    # Get both the allow decision and denial reasons
    allow_result = safe_evaluate("hello/allow", input_data)
    reasons_result = safe_evaluate("hello/denial_reasons", input_data)

    if allow_result is None or reasons_result is None:
        return

    allowed = allow_result.get("result", False)
    reasons = reasons_result.get("result", [])

    # Pretty print the input
    for _key, _value in input_data.items():
        pass

    # Print the decision
    if allowed:
        pass
    else:
        if reasons:
            for _reason in reasons:
                pass
        else:
            pass

In [ ]:
# Test case 1: Unknown role
explain_decision(
    {
        "user": {"role": "guest"},
        "action": "read",
        "resource": "data",
    }
)

In [ ]:
# Test case 2: Missing role
explain_decision(
    {
        "user": {},
        "action": "read",
        "resource": "data",
    }
)

In [ ]:
# Test case 3: Developer attempting forbidden action
explain_decision(
    {
        "user": {"role": "developer"},
        "action": "write",
        "resource": "config",
    }
)

## 5. Batch Evaluation

Let's test multiple scenarios at once to understand the policy behavior across different combinations of roles and actions.

In [ ]:
# Define test scenarios
scenarios = [
    {"user": {"role": "admin"}, "action": "read", "resource": "policy"},
    {"user": {"role": "admin"}, "action": "write", "resource": "policy"},
    {"user": {"role": "admin"}, "action": "delete", "resource": "policy"},
    {"user": {"role": "developer"}, "action": "read", "resource": "policy"},
    {"user": {"role": "developer"}, "action": "write", "resource": "policy"},
    {"user": {"role": "developer"}, "action": "delete", "resource": "policy"},
    {"user": {"role": "guest"}, "action": "read", "resource": "policy"},
    {"user": {"role": "guest"}, "action": "write", "resource": "policy"},
    {"user": {}, "action": "read", "resource": "policy"},
]

# Evaluate all scenarios
results = []
for scenario in scenarios:
    result = safe_evaluate("hello/allow", scenario)
    if result:
        results.append(
            {
                "role": scenario.get("user", {}).get("role", "(none)"),
                "action": scenario.get("action", "(none)"),
                "allowed": result.get("result", False),
            }
        )

# Create DataFrame for easy viewing
df = pd.DataFrame(results)

## 6. Visualization

Let's visualize the policy decisions to better understand access patterns.

In [ ]:
# Create a pivot table for the heatmap
if len(df) > 0:
    pivot_df = (
        df.pivot_table(
            index="role",
            columns="action",
            values="allowed",
            aggfunc="first",
        )
        .fillna(False)
        .astype(int)
    )

    # Create heatmap
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.heatmap(
        pivot_df,
        annot=True,
        cmap="RdYlGn",
        cbar_kws={"label": "Allowed (1) / Denied (0)"},
        fmt="d",
        ax=ax,
    )
    ax.set_title("RBAC Policy Decisions: Role vs Action")
    ax.set_xlabel("Action")
    ax.set_ylabel("Role")
    plt.tight_layout()
    plt.show()

    # Close figure to prevent memory leaks
    plt.close(fig)
else:
    pass

In [ ]:
# Create a bar chart showing allow/deny counts by role
if len(df) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))

    role_counts = df.groupby(["role", "allowed"]).size().unstack(fill_value=0)
    role_counts.columns = ["Denied", "Allowed"]

    role_counts.plot(
        kind="bar",
        stacked=True,
        color=["#E74C3C", "#27AE60"],
        ax=ax,
    )
    ax.set_title("Policy Decisions by Role")
    ax.set_xlabel("Role")
    ax.set_ylabel("Number of Decisions")
    ax.legend(title="Decision")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Close figure to prevent memory leaks
    plt.close(fig)
else:
    pass

## 7. Interactive Experimentation

Now it's your turn! Use the cells below to experiment with different policy inputs.

In [ ]:
# EXERCISE 1: Try different role/action combinations
# Modify the values below and run the cell to see the result

my_input = {
    "user": {"role": "developer"},  # Try: admin, developer, guest, viewer
    "action": "read",  # Try: read, write, delete, update
    "resource": "policy",  # Try: policy, config, data
}

explain_decision(my_input)

In [ ]:
# EXERCISE 2: Create your own test scenarios
# Add more scenarios to the list and run batch evaluation

custom_scenarios = [
    # Add your test cases here
    {"user": {"role": "admin"}, "action": "audit", "resource": "logs"},
    {"user": {"role": "viewer"}, "action": "read", "resource": "reports"},
    # Add more...
]

for scenario in custom_scenarios:
    explain_decision(scenario)

## 8. Working with AI Model Approval Policy

If you have Example 02 running, you can also experiment with the AI model approval policy. This demonstrates more complex governance scenarios.

In [ ]:
# Check if the AI model approval policy is loaded
test_model = {
    "model": {
        "id": "test-model-001",
        "risk_score": 0.3,
        "environment": "staging",
    },
    "compliance": {
        "bias_tested": True,
        "documentation_complete": True,
        "security_reviewed": True,
    },
    "deployment": {
        "environment": "staging",
    },
}

result = safe_evaluate("ai/model_approval/allow", test_model)
if result:
    allowed = result.get("result", False)
else:
    pass

In [ ]:
# Compare low-risk vs high-risk models

# Define compliance status (shared across scenarios)
full_compliance = {
    "bias_tested": True,
    "documentation_complete": True,
    "security_reviewed": True,
}

risk_scenarios = [
    {
        "name": "Low-risk model (staging)",
        "input": {
            "model": {
                "id": "low-risk-001",
                "risk_score": 0.2,
                "environment": "staging",
            },
            "compliance": full_compliance,
            "deployment": {"environment": "staging"},
        },
    },
    {
        "name": "Medium-risk model (staging)",
        "input": {
            "model": {
                "id": "medium-risk-001",
                "risk_score": 0.5,
                "environment": "staging",
            },
            "compliance": full_compliance,
            "deployment": {"environment": "staging"},
        },
    },
    {
        "name": "High-risk model (production, no reviewer)",
        "input": {
            "model": {
                "id": "high-risk-001",
                "risk_score": 0.8,
                "environment": "production",
            },
            "compliance": full_compliance,
            "deployment": {"environment": "production"},
        },
    },
    {
        "name": "High-risk model (production, with reviewer)",
        "input": {
            "model": {
                "id": "high-risk-002",
                "risk_score": 0.8,
                "environment": "production",
            },
            "compliance": full_compliance,
            "deployment": {"environment": "production"},
            "reviewer": {"id": "reviewer-001", "role": "senior_engineer"},
        },
    },
]

for scenario in risk_scenarios:
    result = safe_evaluate("ai/model_approval/allow", scenario["input"])
    if result:
        allowed = result.get("result", False)
        status = "APPROVED" if allowed else "DENIED"
    else:
        pass

## 9. Summary Statistics

Let's generate some summary statistics from our policy evaluations.

In [ ]:
# Generate extended test scenarios for statistics
extended_scenarios = []
roles = ["admin", "developer", "guest", "viewer", "operator"]
actions = ["read", "write", "delete", "update", "execute"]

for role in roles:
    for action in actions:
        extended_scenarios.append(
            {
                "user": {"role": role},
                "action": action,
                "resource": "data",
            }
        )

# Evaluate all scenarios
extended_results = []
for scenario in extended_scenarios:
    result = safe_evaluate("hello/allow", scenario)
    if result:
        extended_results.append(
            {
                "role": scenario["user"]["role"],
                "action": scenario["action"],
                "allowed": result.get("result", False),
            }
        )

if extended_results:
    ext_df = pd.DataFrame(extended_results)

    allowed_pct = ext_df["allowed"].mean() * 100
    denied_pct = (~ext_df["allowed"]).mean() * 100
else:
    pass

In [ ]:
# Visualize approval rates
if extended_results:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Pie chart: Overall allow/deny
    allowed_count = ext_df["allowed"].sum()
    denied_count = (~ext_df["allowed"]).sum()

    axes[0].pie(
        [allowed_count, denied_count],
        labels=["Allowed", "Denied"],
        colors=["#27AE60", "#E74C3C"],
        autopct="%1.1f%%",
        startangle=90,
    )
    axes[0].set_title("Overall Policy Decisions")

    # Bar chart: Approval rate by role
    approval_rates = ext_df.groupby("role")["allowed"].mean()
    approval_rates = approval_rates.sort_values(ascending=True)
    colors = ["#27AE60" if rate > 0.5 else "#E74C3C" for rate in approval_rates]

    approval_rates.plot(
        kind="barh",
        color=colors,
        ax=axes[1],
    )
    axes[1].set_title("Approval Rate by Role")
    axes[1].set_xlabel("Approval Rate")
    axes[1].set_ylabel("Role")
    axes[1].set_xlim(0, 1)

    plt.tight_layout()
    plt.show()

    # Close figure to prevent memory leaks
    plt.close(fig)
else:
    pass

## 10. Cleanup

Always clean up resources to prevent memory leaks, especially in Docker environments.

In [ ]:
# Close all matplotlib figures
plt.close("all")

---

## Next Steps

You've completed the policy experimentation notebook! Here's what to explore next:

1. **Notebook 02**: Governance Visualization - More advanced charts and dashboards
2. **Example Projects**: Try the examples in `/examples/` directory
3. **Custom Policies**: Write your own Rego policies and test them here
4. **OPA Documentation**: [openpolicyagent.org/docs](https://www.openpolicyagent.org/docs/)

### Feedback

Did this notebook help you understand OPA policy evaluation? We'd love your feedback!

- [Submit Feedback](../docs/feedback.md)
- [Report Issues](https://github.com/your-org/acgs2/issues)

---

*ACGS-2 Developer Onboarding - Policy Experimentation Notebook*